# Parser Teks Dokumen dan Pengekstrak Kata Kunci Keuangan (Tahunan)

Notebook ini bertujuan untuk mem-parsing teks dari berbagai jenis dokumen (gambar, TXT, DOCX, PDF) dan kemudian mengekstrak istilah-istilah keuangan spesifik beserta nilainya, dengan fokus pada **data tahun pelaporan terbaru**. Proses ini melibatkan beberapa teknologi dan fitur utama:
- **OCR (Optical Character Recognition)**: Untuk mengekstrak teks dari gambar dan PDF berbasis gambar.
- **Pra-pemrosesan Gambar**: Sebelum OCR, gambar diproses melalui beberapa tahap (konversi ke skala abu, penghilangan derau, binerisasi, dan percobaan pelurusan kemiringan) untuk meningkatkan kualitas OCR.
- **Pemrosesan Paralel untuk PDF**: Halaman PDF yang memerlukan OCR diproses secara paralel untuk mempercepat ekstraksi.
- **Mekanisme Caching**: Hasil parsing PDF disimpan dalam cache untuk menghindari pemrosesan ulang file yang sama jika tidak ada perubahan.
- **Ekstraksi Kata Kunci Bertarget**: Mencari istilah keuangan yang telah ditentukan (dalam Bahasa Indonesia) dan mencoba mengidentifikasi nilai numerik yang berasosiasi dengan tahun pelaporan terbaru yang terdeteksi dalam dokumen.
- **Normalisasi Nilai**: Nilai keuangan yang diekstrak dinormalisasi ke format float.
- **Output JSON**: Hasil akhir ekstraksi kata kunci dan nilainya disajikan dalam format JSON.

Pastikan semua skrip Python pendukung (`parser_gambar.py`, `parser_dokumen_teks.py`, `parser_pdf.py`, `pengekstrak_kata_kunci.py`, `utilitas_cache.py`) berada di direktori yang sama dengan notebook ini atau terinstal dalam lingkungan Python Anda.


In [1]:
# Langkah Pengaturan Awal

# 1. Impor Pustaka dan Modul Kustom
# Pastikan semua skrip Python (.py) yang disebutkan di bawah ini
# berada di direktori yang sama dengan notebook ini.

import os
import json
import nltk

# Impor fungsi-fungsi dari modul-modul utilitas kita
try:
    from SaranaModule.parser_gambar import ekstrak_teks_dari_gambar
    from SaranaModule.parser_dokumen_teks import ekstrak_teks_dari_txt, ekstrak_teks_dari_docx
    from SaranaModule.parser_pdf import ekstrak_teks_dari_pdf # Fungsi ini menggunakan ekstrak_teks_dari_gambar untuk OCR
    from SaranaModule.pengekstrak_kata_kunci import (
        DAFTAR_KATA_KUNCI_KEUANGAN_DEFAULT, # Daftar kata kunci default
        identifikasi_tahun_pelaporan,       # Untuk menemukan tahun dalam dokumen
        ekstrak_data_keuangan_tahunan,      # Fungsi ekstraksi utama yang baru
        format_ke_json,                     # Untuk output JSON
        normalisasi_nilai_keuangan,          # Untuk membersihkan nilai angka (jika ingin diuji terpisah)
        deteksi_pengali_global              # Untuk mendeteksi pengali global seperti 'juta', 'miliar', dll.
    )
    from SaranaModule.utilitas_cache import bersihkan_cache_lama # Opsional, untuk manajemen cache
    print("Modul-modul kustom berhasil diimpor.")
except ImportError as e:
    print(f"Error mengimpor modul kustom: {e}")
    print("Pastikan semua file .py (parser_gambar, parser_dokumen_teks, parser_pdf, pengekstrak_kata_kunci, utilitas_cache) berada di direktori yang sama.")


try:
    nltk.data.find('corpora/wordnet.zip')
    print("Resource NLTK (wordnet) sudah ada.")
except LookupError:
    import ssl
    ssl._create_default_https_context = ssl._create_unverified_context
    print("Resource NLTK (wordnet) tidak ditemukan, mengunduh...")
    nltk.download('wordnet')
    nltk.download('punkt_tab')
    nltk.download('omw-1.4') # wordnet multilingual
    nltk.download('punkt')   # untuk tokenisasi
    nltk.download('stopwords') # untuk stopwords
    print("Resource NLTK (wordnet) sudah terunduh.")

except Exception as e:
     print(f"Error terkait NLTK: {e}")

print("\nPengaturan Selesai. Anda dapat melanjutkan ke sel Konfigurasi.")

Berhasil memuat stopwords Bahasa Indonesia.
WordNetLemmatizer berhasil diinisialisasi.
Tokenizer 'punkt' tampaknya tersedia.
Modul-modul kustom berhasil diimpor.
Resource NLTK (wordnet) sudah ada.

Pengaturan Selesai. Anda dapat melanjutkan ke sel Konfigurasi.


## Penjelasan Fitur Utama

Sebelum melanjutkan ke konfigurasi, berikut adalah ringkasan singkat tentang beberapa fitur utama yang digunakan dalam notebook ini:

*   **Pra-pemrosesan Gambar untuk OCR**: Jika dokumen Anda adalah gambar atau PDF yang berisi halaman gambar, kualitas OCR sangat penting. Modul `parser_gambar.py` kini menyertakan langkah-langkah seperti konversi ke skala abu, penghilangan derau (noise), dan binerisasi (mengubah gambar menjadi hitam-putih) untuk meningkatkan akurasi Tesseract OCR. Implementasi dasar untuk pelurusan kemiringan (deskewing) juga ada, meskipun mungkin memerlukan penyesuaian lebih lanjut untuk kasus yang kompleks.
*   **Pemrosesan Paralel untuk PDF**: Untuk mempercepat ekstraksi teks dari PDF yang memiliki banyak halaman berbasis gambar (yang memerlukan OCR), `parser_pdf.py` menggunakan `ThreadPoolExecutor`. Ini memungkinkan beberapa halaman diproses secara bersamaan, mengurangi waktu tunggu total.
*   **Caching Hasil Parsing**: Untuk menghindari pemrosesan ulang file PDF yang sama berulang kali (yang bisa memakan waktu), `parser_pdf.py` kini terintegrasi dengan mekanisme caching (`utilitas_cache.py`). Hasil ekstraksi teks dari sebuah PDF akan disimpan dalam cache (default di direktori `.cache_parser_dokumen`). Jika Anda memproses PDF yang sama lagi dan file tersebut tidak berubah (berdasarkan path dan timestamp modifikasi terakhir), hasilnya akan diambil dari cache, yang jauh lebih cepat. Anda bisa membersihkan cache ini secara manual atau menggunakan fungsi `bersihkan_cache_lama` (jika ingin diimplementasikan lebih lanjut).
*   **Logika Ekstraksi Nilai Berbasis Tahun**: Fungsi `ekstrak_data_keuangan_tahunan` dalam `pengekstrak_kata_kunci.py` dirancang untuk pertama-tama mengidentifikasi tahun pelaporan utama dalam dokumen. Kemudian, saat mencari nilai untuk kata kunci keuangan, ia akan mencoba memprioritaskan angka yang berasosiasi dengan tahun pelaporan tersebut dan membedakannya dari angka untuk tahun sebelumnya, jika keduanya muncul berdekatan.


In [ ]:
# --- Konfigurasi Pengguna ---

# 1. Tentukan path ke dokumen Anda
# PENTING: Ganti nilai variabel `path_dokumen` di bawah ini dengan path aktual ke dokumen yang ingin Anda proses.
# Contoh untuk Linux/macOS: path_dokumen = "/home/pengguna/dokumen/laporan_keuangan_2023.pdf"
# Contoh untuk Windows: path_dokumen = r"C:\Users\Pengguna\Documents\LaporanKeuangan2023.docx"
path_dokumen = "train_documents/astra_lapkeu.pdf"  # <--- !!! GANTI INI DENGAN PATH FILE ANDA !!!

# 2. Definisikan atau Modifikasi Kata Kunci yang Akan Diekstrak
# `konfigurasi_kata_kunci_target` adalah list kamus (dictionary).
# Setiap kamus harus memiliki:
#    - 'kata_dasar': Nama kanonis untuk kata kunci tersebut (misalnya, "Laba Bersih"). Ini akan menjadi kunci dalam output JSON.
#    - 'variasi': List berisi berbagai cara penulisan atau sinonim kata kunci tersebut yang mungkin muncul di dokumen.
#
# Anda bisa menggunakan daftar default yang diimpor (`DAFTAR_KATA_KUNCI_KEUANGAN_DEFAULT`)
# atau membuat/memodifikasi daftar Anda sendiri di bawah ini.
#
# Untuk menggunakan daftar default:
from SaranaModule.pengekstrak_kata_kunci import DAFTAR_KATA_KUNCI_KEUANGAN_DEFAULT
konfigurasi_kata_kunci_target = DAFTAR_KATA_KUNCI_KEUANGAN_DEFAULT

# 3. (Opsional) Konfigurasi Direktori Cache untuk PDF Parser
# Jika Anda ingin parser PDF menggunakan direktori cache selain default (`.cache_parser_dokumen`),
# Anda bisa menentukan path-nya di sini. Jika tidak, biarkan `None` untuk menggunakan default.
direktori_cache_pdf_kustom = ".cache_parsing_dokumen"

# --- Akhir Konfigurasi Pengguna ---

# Validasi awal konfigurasi
if 'path_dokumen' not in locals() or path_dokumen == "MASUKKAN_PATH_DOKUMEN_ANDA_DI_SINI.pdf":
    print("PERINGATAN: 'path_dokumen' belum diatur atau masih menggunakan nilai placeholder.")
    print("Mohon perbarui variabel 'path_dokumen' di atas dengan path ke berkas yang ingin Anda proses.")
elif not os.path.exists(path_dokumen):
    print(f"ERROR: Dokumen tidak ditemukan pada path yang ditentukan: {path_dokumen}")
    print("Mohon periksa kembali 'path_dokumen' dan pastikan berkas tersebut ada.")
else:
    print(f"Konfigurasi dimuat. Dokumen yang akan diproses: {path_dokumen}")
    print(f"Kata kunci yang akan dicari: {[item['kata_dasar'] for item in konfigurasi_kata_kunci_target]}")
    if direktori_cache_pdf_kustom:
        print(f"Direktori cache PDF kustom diatur ke: {direktori_cache_pdf_kustom}")



Konfigurasi dimuat. Dokumen yang akan diproses: train_documents
Kata kunci yang akan dicari: ['Jumlah aset lancar', 'Jumlah aset tidak lancar', 'Jumlah aset', 'Jumlah liabilitas jangka pendek', 'Jumlah liabilitas jangka panjang', 'Jumlah liabilitas', 'Jumlah ekuitas', 'Pendapatan bersih', 'Beban pokok pendapatan', 'Laba bruto', 'Laba sebelum pajak penghasilan', 'Laba tahun berjalan', 'Beban penjualan', 'Beban umum dan administrasi', 'Penghasilan bunga', 'Biaya keuangan', 'Keuntungan selisih kurs, bersih', 'Penghasilan dividen', 'Penghasilan lain-lain, bersih', 'Beban pajak penghasilan']
Direktori cache PDF kustom diatur ke: .cache_parsing_dokumen


In [ ]:
# Langkah ini akan mengekstrak seluruh teks dari dokumen yang telah Anda tentukan.
# Proses ini mungkin memakan waktu, terutama untuk PDF besar atau gambar yang memerlukan OCR.

teks_hasil_ekstraksi = ""

# Pastikan path_dokumen telah dikonfigurasi dengan benar sebelum melanjutkan
if 'path_dokumen' not in locals() or path_dokumen == "MASUKKAN_PATH_DOKUMEN_ANDA_DI_SINI.pdf" or not os.path.exists(path_dokumen):
    if 'path_dokumen' in locals() and path_dokumen == "MASUKKAN_PATH_DOKUMEN_ANDA_DI_SINI.pdf":
        pesan_error_parsing = "Error: 'path_dokumen' masih menggunakan nilai placeholder. Silakan perbarui di sel Konfigurasi."
    elif 'path_dokumen' not in locals():
         pesan_error_parsing = "Error: 'path_dokumen' tidak terdefinisi. Silakan definisikan di sel Konfigurasi."
    else: # Berkas tidak ada
        pesan_error_parsing = f"Error: Dokumen tidak ditemukan pada path '{path_dokumen}'. Mohon verifikasi path di sel Konfigurasi."
    print(pesan_error_parsing)
    teks_hasil_ekstraksi = pesan_error_parsing # Simpan pesan error untuk sel berikutnya
else:
    print(f"Memulai parsing untuk dokumen: {path_dokumen}...")
    nama_file = os.path.basename(path_dokumen)
    ekstensi_file = os.path.splitext(nama_file)[1].lower()
    print(f"Tipe berkas terdeteksi: {ekstensi_file}")
    
    try:
        if ekstensi_file == '.pdf':
            # Menggunakan fungsi ekstrak_teks_dari_gambar untuk OCR, dan direktori cache jika ada
            # Pastikan variabel 'direktori_cache_pdf_kustom' ada dari sel konfigurasi
            dir_cache = direktori_cache_pdf_kustom if 'direktori_cache_pdf_kustom' in locals() else None
            teks_hasil_ekstraksi = ekstrak_teks_dari_pdf(path_dokumen, ekstrak_teks_dari_gambar, direktori_cache_kustom=dir_cache)
        elif ekstensi_file in ['.jpg', '.jpeg', '.png', '.tiff', '.bmp', '.gif']:
            teks_hasil_ekstraksi = ekstrak_teks_dari_gambar(path_dokumen)
        elif ekstensi_file == '.txt':
            teks_hasil_ekstraksi = ekstrak_teks_dari_txt(path_dokumen)
        elif ekstensi_file == '.docx':
            teks_hasil_ekstraksi = ekstrak_teks_dari_docx(path_dokumen)
        else:
            teks_hasil_ekstraksi = (f"Error: Tipe berkas tidak didukung: {ekstensi_file}. "
                                   f"Tipe yang didukung: PDF, JPG, JPEG, PNG, TIFF, BMP, GIF, TXT, DOCX.")
    except Exception as e:
        teks_hasil_ekstraksi = f"Error selama parsing dokumen '{nama_file}': {str(e)}"

# Tampilkan status dan cuplikan hasil ekstraksi
if "Error:" not in teks_hasil_ekstraksi:
    print(f"Parsing dokumen '{nama_file}' selesai.")
    print(f"Total karakter yang diekstrak: {len(teks_hasil_ekstraksi)}")
    print("\n--- Cuplikan Teks Hasil Ekstraksi (500 karakter pertama) ---")
    print(teks_hasil_ekstraksi[:1000000] + "..." if len(teks_hasil_ekstraksi) > 500 else teks_hasil_ekstraksi)
    print("--- Akhir Cuplikan ---")
else:
    # Pesan error sudah dicetak di atas jika path_dokumen tidak valid
    # Cetak pesan error jika berasal dari proses parsing itu sendiri
    if not ('path_dokumen' in locals() and path_dokumen == "MASUKKAN_PATH_DOKUMEN_ANDA_DI_SINI.pdf") and        not ('path_dokumen' in locals() and not os.path.exists(path_dokumen)):
        print(teks_hasil_ekstraksi)


In [ ]:
pengali_dokumen = 1.0
print_output_pengali = []

if 'teks_hasil_ekstraksi' in locals() and isinstance(teks_hasil_ekstraksi, str) and not "Error:" in teks_hasil_ekstraksi:
    pengali_dokumen = deteksi_pengali_global(teks_hasil_ekstraksi)
    print_output_pengali.append(f"Pengali dokumen yang terdeteksi: {pengali_dokumen}")
elif 'teks_hasil_ekstraksi' in locals() and isinstance(teks_hasil_ekstraksi, str) and "Error:" in teks_hasil_ekstraksi:
    error_msg_summary = teks_hasil_ekstraksi[:200].replace('"', '\"').replace('\\', '\\\\')
    print_output_pengali.append(f"Tidak menjalankan deteksi pengali global karena ada error pada ekstraksi teks: {error_msg_summary}")
else:
    print_output_pengali.append("Tidak menjalankan deteksi pengali global karena 'teks_hasil_ekstraksi' tidak valid atau tidak ditemukan.")

# Pastikan variabel ini ada, meskipun terjadi error di atas, untuk sel berikutnya
if 'pengali_dokumen' not in locals():
    pengali_dokumen = 1.0
    print_output_pengali.append("Variabel 'pengali_dokumen' diinisialisasi ke default (1.0) karena tidak terdefinisi sebelumnya.")

# Print semua pesan terkait deteksi pengali
for msg in print_output_pengali:
    print(msg)

# Langkah Ekstraksi Kata Kunci
kamus_hasil_ekstraksi = {}

if 'teks_hasil_ekstraksi' not in locals() or not teks_hasil_ekstraksi:
    print("Error: Variabel 'teks_hasil_ekstraksi' tidak ditemukan atau kosong. Sel parsing dokumen mungkin belum dijalankan atau gagal.")
    kamus_hasil_ekstraksi = {"error_pra_ekstraksi": "Variabel teks_hasil_ekstraksi tidak ada atau kosong."}
elif "Error:" in teks_hasil_ekstraksi: # Jika ada pesan error dari langkah parsing
    print("\nMelewati ekstraksi kata kunci karena terjadi error pada langkah parsing dokumen.")
    print(f"Detail Error Parsing: {teks_hasil_ekstraksi}")
    kamus_hasil_ekstraksi = {"error_parsing_sebelumnya": teks_hasil_ekstraksi}
elif not teks_hasil_ekstraksi.strip():
    print("\nTeks yang diekstrak kosong atau hanya berisi spasi putih. Tidak ada kata kunci untuk dicari.")
    kamus_hasil_ekstraksi = {"info": "Teks yang diekstrak kosong."}
else:
    try:
        # Pastikan konfigurasi_kata_kunci_target terdefinisi dari sel Konfigurasi
        if 'konfigurasi_kata_kunci_target' not in locals() or not konfigurasi_kata_kunci_target:
            pesan_error_konfig = "Error: 'konfigurasi_kata_kunci_target' tidak terdefinisi atau kosong. Mohon periksa sel Konfigurasi."
            print(pesan_error_konfig)
            kamus_hasil_ekstraksi = {"error_konfigurasi": pesan_error_konfig}
        else:
            print("\nMemulai proses ekstraksi kata kunci dan nilai tahunan...")
            # Menggunakan fungsi ekstraksi data keuangan tahunan yang baru
            kamus_hasil_ekstraksi = ekstrak_data_keuangan_tahunan(teks_hasil_ekstraksi, konfigurasi_kata_kunci_target, pengali_global=pengali_dokumen)
            
            print("\n--- Kata Kunci dan Nilai Hasil Ekstraksi (Kamus Python) ---")
            if "error" not in kamus_hasil_ekstraksi and "error_konfigurasi" not in kamus_hasil_ekstraksi:
                if kamus_hasil_ekstraksi:
                    for kunci, nilai in kamus_hasil_ekstraksi.items():
                        print(f"- {kunci}: {nilai if nilai is not None else 'Tidak ditemukan'}")
                else:
                    print("Tidak ada kata kunci yang berhasil diekstrak atau dikonfigurasi.")
            else: # Cetak error jika ada dari proses ekstraksi itu sendiri
                 print(f"Error atau masalah dalam hasil ekstraksi: {kamus_hasil_ekstraksi}")

    except Exception as e:
        pesan_error_ekstraksi = f"Error selama proses ekstraksi kata kunci: {str(e)}"
        print(pesan_error_ekstraksi)
        kamus_hasil_ekstraksi = {"error_runtime_ekstraksi": pesan_error_ekstraksi}


Pengali dokumen yang terdeteksi: 1000000000.0

Memulai proses ekstraksi kata kunci dan nilai tahunan...

--- Kata Kunci dan Nilai Hasil Ekstraksi (Kamus Python) ---
- Jumlah aset lancar: 19238000000.0
- Jumlah aset tidak lancar: 81765000000.0
- Jumlah aset: 19238000000.0
- Jumlah liabilitas jangka pendek: 14300000000.0
- Jumlah liabilitas jangka panjang: 1989000000.0
- Jumlah liabilitas: 14300000000.0
- Jumlah ekuitas: 84714000000.0
- Pendapatan bersih: 108249000000.0
- Beban pokok pendapatan: -97738000000.0
- Laba bruto: 10511000000.0
- Laba sebelum pajak penghasilan: 22136000000.0
- Laba tahun berjalan: 21661000000.0
- Beban penjualan: -5298000000.0
- Beban umum dan administrasi: -4701000000.0
- Penghasilan bunga: 322000000000.0
- Biaya keuangan: -119000000000.0
- Keuntungan selisih kurs, bersih: 1000000000.0
- Penghasilan dividen: 19918000000.0
- Penghasilan lain-lain, bersih: 1502000000.0
- Beban pajak penghasilan: -475000000000.0


In [10]:
# Langkah terakhir adalah memformat kamus hasil ekstraksi ke dalam format JSON
# untuk kemudahan pembacaan atau integrasi lebih lanjut.

output_json_final = ""
if 'kamus_hasil_ekstraksi' not in locals() or not kamus_hasil_ekstraksi:
    print("Error: 'kamus_hasil_ekstraksi' tidak ditemukan atau kosong. Ekstraksi kata kunci mungkin gagal, dilewati, atau tidak menghasilkan apa-apa.")
    kamus_hasil_ekstraksi_untuk_json = {"error_kritis": "kamus_hasil_ekstraksi tidak tersedia untuk format JSON."}
    if 'kamus_hasil_ekstraksi' in locals() and not kamus_hasil_ekstraksi:
        kamus_hasil_ekstraksi_untuk_json = {"info": "Tidak ada data yang diekstrak untuk diformat ke JSON."}
else:
    kamus_hasil_ekstraksi_untuk_json = kamus_hasil_ekstraksi

output_json_final = format_ke_json(kamus_hasil_ekstraksi_untuk_json)
print("\n--- Output Final (JSON) ---")
print(output_json_final)

# Menyimpan output JSON ke direktori "output_json"
nama_file_output_json = "hasil_ekstraksi_data.json"
if 'path_dokumen' in locals() and path_dokumen != "MASUKKAN_PATH_DOKUMEN_ANDA_DI_SINI.pdf" and os.path.exists(path_dokumen):
    nama_dasar_dokumen = os.path.splitext(os.path.basename(path_dokumen))[0]
    nama_file_output_json = f"hasil_ekstraksi_{nama_dasar_dokumen}.json"
    output_dir = "OutputSarana"  # Direktori output default
    os.makedirs(output_dir, exist_ok=True)
    full_output_path = os.path.join(output_dir, nama_file_output_json)
    with open(full_output_path, "w", encoding="utf-8") as f:
        f.write(output_json_final)
    print(f"\nMenyimpan hasil ekstraksi ke berkas JSON: {full_output_path}")
else:
    print("\nInfo: Penyimpanan ke berkas JSON dilewati karena path_dokumen tidak valid atau belum diatur.")


--- Output Final (JSON) ---
{
    "Jumlah aset lancar": 19238000000.0,
    "Jumlah aset tidak lancar": 81765000000.0,
    "Jumlah aset": 19238000000.0,
    "Jumlah liabilitas jangka pendek": 14300000000.0,
    "Jumlah liabilitas jangka panjang": 1989000000.0,
    "Jumlah liabilitas": 14300000000.0,
    "Jumlah ekuitas": 84714000000.0,
    "Pendapatan bersih": 108249000000.0,
    "Beban pokok pendapatan": -97738000000.0,
    "Laba bruto": 10511000000.0,
    "Laba sebelum pajak penghasilan": 22136000000.0,
    "Laba tahun berjalan": 21661000000.0,
    "Beban penjualan": -5298000000.0,
    "Beban umum dan administrasi": -4701000000.0,
    "Penghasilan bunga": 322000000000.0,
    "Biaya keuangan": -119000000000.0,
    "Keuntungan selisih kurs, bersih": 1000000000.0,
    "Penghasilan dividen": 19918000000.0,
    "Penghasilan lain-lain, bersih": 1502000000.0,
    "Beban pajak penghasilan": -475000000000.0
}

Menyimpan hasil ekstraksi ke berkas JSON: OutputSarana/hasil_ekstraksi_train_docume

## Catatan Akhir: Efisiensi, Keterbatasan, dan Pengembangan Lanjutan

Notebook ini menyediakan alur kerja yang komprehensif untuk parsing dokumen dan ekstraksi informasi keuangan dasar. Namun, ada beberapa hal yang perlu diperhatikan:

*   **Efisiensi Pemrosesan**:
    *   **PDF Besar**: Seperti yang disebutkan, PDF besar dengan banyak halaman gambar bisa lambat karena OCR. Fitur **OCR Paralel** yang diimplementasikan di `parser_pdf.py` membantu mengurangi waktu tunggu.
    *   **Caching**: Mekanisme **caching** untuk `parser_pdf.py` (disimpan di `.cache_parser_dokumen` secara default) akan sangat membantu jika Anda sering memproses ulang dokumen yang sama, karena hasil parsing akan diambil dari cache jika file tidak berubah.
    *   **Pra-pemrosesan Gambar**: Langkah ini penting untuk akurasi OCR, tetapi juga menambah waktu pemrosesan untuk setiap gambar/halaman gambar.

*   **Akurasi Ekstraksi Kata Kunci dan Nilai**:
    *   **Logika Tahun Terbaru**: `pengekstrak_kata_kunci.py` kini mencoba mengidentifikasi tahun pelaporan dan memprioritaskan nilai yang berasosiasi dengan tahun tersebut, serta membedakannya dari nilai tahun sebelumnya. Akurasi logika ini sangat bergantung pada konsistensi format tabel dan layout dalam dokumen. Mungkin memerlukan penyesuaian regex lebih lanjut untuk berbagai format laporan keuangan.
    *   **Variasi Kata Kunci**: Keberhasilan ekstraksi juga bergantung pada seberapa komprehensif daftar `variasi` untuk setiap `kata_dasar` dalam `konfigurasi_kata_kunci_target`.
    *   **Normalisasi Nilai**: Fungsi `normalisasi_nilai_keuangan` menangani format umum Indonesia, tetapi format yang sangat tidak standar mungkin memerlukan penyesuaian.
    *   **Konteks**: Ekstraktor saat ini menggunakan konteks kalimat dan kedekatan dengan tahun. Untuk kasus yang sangat ambigu, pemahaman struktur tabel atau elemen visual mungkin diperlukan (di luar cakupan saat ini).

*   **Keterbatasan Bahasa Indonesia di NLTK**:
    *   **Stopwords**: `pengekstrak_kata_kunci.py` mencoba menggunakan stopwords Bahasa Indonesia dari NLTK. Pastikan resource ini terinstal (`nltk.download('stopwords')`).
    *   **Lemmatization/Stemming**: `WordNetLemmatizer` NLTK tidak dioptimalkan untuk Bahasa Indonesia. Untuk hasil yang lebih baik dalam normalisasi kata, pertimbangkan untuk mengintegrasikan stemmer khusus Bahasa Indonesia seperti PySastrawi (memerlukan instalasi terpisah). Saat ini, keakuratan pencocokan lebih bergantung pada variasi eksplisit yang disediakan.

*   **Pengembangan Lanjutan yang Mungkin Dilakukan**:
    *   Integrasi stemmer Bahasa Indonesia.
    *   Pengembangan logika yang lebih canggih untuk memahami struktur tabel dalam dokumen.
    *   Pelatihan model Machine Learning kustom untuk klasifikasi teks atau Named Entity Recognition (NER) pada dokumen keuangan untuk identifikasi entitas dan nilai yang lebih robust.
    *   Antarmuka pengguna grafis (GUI) atau aplikasi web di atas logika ini.
